<a href="https://colab.research.google.com/github/AlessioLampis/AlessioLampis/blob/master/Copy_of_Sound_classification_by_Alessio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###This fist part connects the google drive account and loads the urbansound8k data in google collab

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Sound classification

Data Analysis

In [0]:
#import all necessary libraries
import os
import pandas as pd
import IPython as IP
import struct
import matplotlib.pyplot as plt
import numpy as np
import librosa
#import helpers
#from include import helpers

In [5]:
#set the path to the original dataset
us8k_path = 'gdrive/My Drive/urbansound8k'
print(us8k_path)
# Global settings
metadata_path = us8k_path + '/UrbanSound8K.csv'
print(metadata_path)


gdrive/My Drive/urbansound8k
gdrive/My Drive/urbansound8k/UrbanSound8K.csv


In [6]:
# Load metadata as a Pandas dataframe
metadata = pd.read_csv(metadata_path)

# Examine dataframe's head
metadata.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [7]:
# Class distribution
metadata['class'].value_counts()

jackhammer          1000
drilling            1000
air_conditioner     1000
engine_idling       1000
street_music        1000
dog_bark            1000
children_playing    1000
siren                929
car_horn             429
gun_shot             374
Name: class, dtype: int64

In [0]:
def extract_features(file_name):

        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40)
        mfccs_scaled = np.mean(mfccs.T,axis=0)
       	return mfccs_scaled


features = []

# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():
    
    file_name = us8k_path + '/' + 'fold'+ str(row["fold"]) + '/' + str(row["slice_file_name"])
    class_label = row["class"]
    fold = row["fold"]
    data = extract_features(file_name)
    
    features.append([data, class_label,fold])

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label','fold'])

In [0]:
#define a function that extract the features
#audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast');


In [0]:
#extract the Mel coefficients
mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40); #the number of elements of the MFCCS is set in n_mfcc 
#Why does it calculate a mean value?
mfccsscaled = np.mean(mfccs.T,axis=0); #mfccs.T computes the transpose!! Let's scale the MFCCs such that each coefficient dimension has zero mean 
len(mfccsscaled)


Converting the data and labels then splitting the dataset


In [0]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

